# Review Stockout Thresholds

This notebook has the objective of reviwing the results from filtering the monthly sales data based on the stockout thresholds chosen. 
To accomplish this, the notebook: 
1. Reports on the number of rows per department over time - only considering initialized items.
2. Reports on the number of rows after filtering for stockouts.
3. Provides with summary statisticks for the stockout vs non-stockout distributions.
4. Provides with plots for the stockout vs non-stockout distributions.

**Note:** To simplify the analysis I will define "over time" as "per year".


Note to self: Before I tackle this I need to know that the filter for "initialized items" is working as expected. I will put this on hold as I prioritize said filter.


In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import ks_2samp
import ruptures as rpt
import sys
import os

# --- Imports from project root ---
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "..",))
sys.path.append(project_root)
from src.core.clients.bigquery import BigQueryClient

# --- Plotting style ---
sns.set(style='whitegrid', context='notebook')

In [ ]:
# --- Load Data From BigQuery ---
from src.core.clients.bigquery import BigQueryClient

bq_client = BigQueryClient()

sales_query = """
    SELECT 
        tgt.ctx_date_month,
        tgt.ctx_cat_id, 
        tgt.ctx_dept_id, 
        tgt.ctx_item_id,
        tgt.tgt_monthly_sales,
        tgt.tgt_monthly_sales_sum_3_next_months,
        stckt.is_stockout_tgt
    FROM `porygon-pipelines.walmart_targets_and_filters.target_store_item_monthly` tgt
    LEFT JOIN `porygon-pipelines.walmart_targets_and_filters.stockout_filter` stckt
        ON tgt.ctx_item_id = stckt.ctx_item_id
        AND tgt.ctx_store_id = stckt.ctx_store_id
        AND tgt.ctx_date_month = stckt.ctx_date_month
    WHERE 
        tgt.ctx_store_id = 'CA_1'
"""


# Extract data
df = bq_client.load_from_query(sales_query)

INFO:src.core.clients.bigquery:BigQuery client initialized for project: porygon-pipelines
/Users/anapreciado/Desktop/porygon-demand-forecasting/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
INFO:src.core.clients.bigquery:Loaded 195136 rows from custom query.


Full dataframe shape: (195136, 7)
Initialized items dataframe shape: (185989, 7)


In [9]:
# --- Data Preparation ---
# Define target name to analyze:
target_name = "tgt_monthly_sales_sum_3_next_months"

# Create a df that only includes initialized items
df_init = df[~df['is_stockout_tgt'].isna()]

# Print summary dataframe shapes:
print(f"Full dataframe shape: {df.shape}")
print(f"Initialized items dataframe shape: {df_init.shape}")

# Create column to denote year.
df_init["year"] = pd.DatetimeIndex(df_init['ctx_date_month']).year

Full dataframe shape: (195136, 7)
Initialized items dataframe shape: (185989, 7)


/var/folders/mq/1yp0hqvn7mg6cbdtcfjwn9fm0000gn/T/ipykernel_50639/3273706823.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init["year"] = pd.DatetimeIndex(df_init['ctx_date_month']).year


In [16]:
df[df['is_stockout_tgt'].isna()]

,ctx_date_month,ctx_cat_id,ctx_dept_id,ctx_item_id,tgt_monthly_sales,tgt_monthly_sales_sum_3_next_months,is_stockout_tgt
0,2016-04-01,FOODS,FOODS_1,FOODS_1_001,29,<NA>,<NA>
1,2016-04-01,FOODS,FOODS_1,FOODS_1_010,7,<NA>,<NA>
2,2016-04-01,FOODS,FOODS_1,FOODS_1_017,24,<NA>,<NA>
3,2016-03-01,FOODS,FOODS_1,FOODS_1_019,225,217,<NA>
4,2016-04-01,FOODS,FOODS_1,FOODS_1_021,36,<NA>,<NA>
...,...,...,...,...,...,...,...
186666,2016-02-01,HOBBIES,HOBBIES_2,HOBBIES_2_136,0,0,<NA>
186729,2016-02-01,HOUSEHOLD,HOUSEHOLD_1,HOUSEHOLD_1_436,14,0,<NA>
186822,2016-02-01,HOUSEHOLD,HOUSEHOLD_2,HOUSEHOLD_2_235,2,4,<NA>
186859,2016-02-01,HOUSEHOLD,HOUSEHOLD_2,HOUSEHOLD_2_323,1,0,<NA>


# 0.1 Report on the numberof prows per item department

This isonly considering initialized items. These summary statistics I want to see them using both summary statistics and plots.

In [ ]:
# Group by year and department, count months per group
grouped = (
    df_init
    .groupby(["year", "ctx_dept_id"])
    .agg(month_count=("ctx_date_month", "count"))
    .reset_index()
)

# Pivot to get years as rows and departments as columns
dept_row_count_per_year = grouped.pivot(
    index="year",
    columns="ctx_dept_id",
    values="month_count"
)

# Note that Since
dept_row_count_per_year

ctx_dept_id,FOODS_1,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2
year,,,,,,,
2011,2592,4776,9876,4992,1788,6384,6180
2012,2592,4776,9876,4992,1788,6384,6180
2013,2592,4776,9876,4992,1788,6384,6180
2014,2592,4776,9876,4992,1788,6384,6180
2015,2592,4776,9876,4992,1788,6384,6180
2016,216,398,823,416,149,532,515
